In [14]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [15]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
from torchvision import models, transforms
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from PIL import Image


# -------------------------------
# CONFIGURATION (EDIT THESE PATHS)
# -------------------------------
VIDEO_DIR = "/content/drive/My Drive/surgeon_profiling_fingerprint/data/Suturing/video/"      # JIGSAWS .avi files
LABEL_DIR = "/content/drive/My Drive/surgeon_profiling_fingerprint/data/Suturing/transcriptions/"      # CSVs with gesture annotations
OUTPUT_PATH = "/content/drive/My Drive/surgeon_profiling_fingerprint/data/jigsaws_full_processed.npz"
#SURGEON_MAP = {"B": 0, "C": 1, "D": 2}  # adjust per dataset
SURGEON_MAP = {chr(c): i for i, c in enumerate(range(ord("B"), ord("J")))}  # B to I

LANG_TEXT = "suturing task"
SEQ_LEN = 5
FRAME_SKIP = 5             # sample every 5th frame


# -------------------------------
# Load Models
# -------------------------------
resnet = models.resnet18(pretrained=True)
resnet.eval()
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert = BertModel.from_pretrained("bert-base-uncased")
bert.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

def get_language_embedding(text):
    tokens = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = bert(**tokens)
    return outputs.last_hidden_state[:, 0, :].squeeze(0).numpy()

def extract_frame_features(video_path):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_indices = []
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret: break
        if frame_idx % FRAME_SKIP == 0:
            image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            x = transform(image).unsqueeze(0)
            with torch.no_grad():
                feat = resnet(x).squeeze().numpy()
            features.append(feat)
            frame_indices.append(frame_idx)
        frame_idx += 1
    cap.release()
    print(f"  📽️ Extracted {len(features)} frames from {video_path}")
    return np.stack(features), frame_indices



def align_gestures(label_path, frame_indices):
    gestures = np.full(len(frame_indices), 15, dtype=int)
    ranges = []

    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 3:
                start, end, g_label = int(parts[0]), int(parts[1]), parts[2]
                ranges.append((start, end, int(g_label[1:]) - 1))  # G1 → 0

    print("  Gesture label ranges (start–end–G):")
    for r in ranges[:5]:
        print("   ", r)

    for i, frame in enumerate(frame_indices):
        for start, end, g in ranges:
            if start <= frame <= end:
                gestures[i] = g
                break
    print(f"  First 20 frame indices: {frame_indices[:20]}")
    print(f"  First 20 gesture labels: {gestures[:20]}")

    return gestures




def process_trial(video_file, lang_embed):
    trial_name = os.path.splitext(video_file)[0]
    parts = trial_name.split("_")
    if len(parts) < 2:
        print(f"Skipping malformed filename: {trial_name}")
        return []

    surgeon_code = parts[1][0]
    surgeon_id = SURGEON_MAP.get(surgeon_code, -1)
    print(surgeon_id)
    if surgeon_id == -1:
        print(f"Unknown surgeon code in {video_file}")
        return []

    video_path = os.path.join(VIDEO_DIR, video_file)
    # Extract "Suturing_B001" from "Suturing_B001_capture1"
    base_trial_id = "_".join(trial_name.split("_")[:2])
    label_path = os.path.join(LABEL_DIR, base_trial_id + ".txt")
    print(f"\n🔍 Processing video: {video_file}")
    print(f"→ Using label file: {label_path}")


    if not os.path.exists(label_path):
        print(f"Missing label file for {trial_name}")
        return []

    vision, frame_indices = extract_frame_features(video_path)
    gestures = align_gestures(label_path, frame_indices)

    samples = []
    for i in range(0, len(vision) - SEQ_LEN + 1, SEQ_LEN):
        samples.append({
            "vision_feat": vision[i:i+SEQ_LEN].mean(axis=0),
            "lang_feat": lang_embed,
            "gesture_seq": gestures[i:i+SEQ_LEN],
            "surgeon_id": surgeon_id
        })
    print(f"  ✅ Appended {len(samples)} sequences for {video_file}")
    print(f"    First gesture seq: {samples[0]['gesture_seq']}" if samples else "    No valid samples.")
    return samples

def main():
    lang_embed = get_language_embedding(LANG_TEXT)
    all_samples = []

    for video_file in tqdm(sorted(os.listdir(VIDEO_DIR))):
        if not video_file.endswith(".avi"): continue
        trial_samples = process_trial(video_file, lang_embed)
        all_samples.extend(trial_samples)

    vision_feats = np.stack([s["vision_feat"] for s in all_samples])
    lang_feats = np.stack([s["lang_feat"] for s in all_samples])
    gesture_seqs = np.stack([s["gesture_seq"] for s in all_samples])
    surgeon_ids = np.array([s["surgeon_id"] for s in all_samples])

    np.savez_compressed(OUTPUT_PATH,
        vision_feat=vision_feats,
        lang_feat=lang_feats,
        gesture_seq=gesture_seqs,
        surgeon_id=surgeon_ids
    )
    print(f"Saved {len(surgeon_ids)} samples to {OUTPUT_PATH}")

if __name__ == "__main__":
    main()


  0%|          | 0/78 [00:00<?, ?it/s]

0

🔍 Processing video: Suturing_B001_capture1.avi
→ Using label file: /content/drive/My Drive/surgeon_profiling_fingerprint/data/Suturing/transcriptions/Suturing_B001.txt


  1%|▏         | 1/78 [00:27<34:56, 27.22s/it]

  📽️ Extracted 1128 frames from /content/drive/My Drive/surgeon_profiling_fingerprint/data/Suturing/video/Suturing_B001_capture1.avi
  Gesture label ranges (start–end–G):
    (80, 219, 0)
    (220, 370, 4)
    (371, 590, 7)
    (591, 660, 1)
    (661, 954, 2)
  First 20 frame indices: [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]
  First 20 gesture labels: [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15  0  0  0  0]
  ✅ Appended 225 sequences for Suturing_B001_capture1.avi
    First gesture seq: [15 15 15 15 15]
0

🔍 Processing video: Suturing_B001_capture2.avi
→ Using label file: /content/drive/My Drive/surgeon_profiling_fingerprint/data/Suturing/transcriptions/Suturing_B001.txt


  3%|▎         | 2/78 [00:54<34:28, 27.22s/it]

...

 99%|█████████▊| 77/78 [20:16<00:18, 18.87s/it]

  📽️ Extracted 756 frames from /content/drive/My Drive/surgeon_profiling_fingerprint/data/Suturing/video/Suturing_I005_capture1.avi
  Gesture label ranges (start–end–G):
    (73, 151, 0)
    (152, 241, 4)
    (242, 518, 7)
    (519, 658, 1)
    (659, 1198, 2)
  First 20 frame indices: [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]
  First 20 gesture labels: [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15  0  0  0  0  0]
  ✅ Appended 151 sequences for Suturing_I005_capture1.avi
    First gesture seq: [15 15 15 15 15]
7

🔍 Processing video: Suturing_I005_capture2.avi
→ Using label file: /content/drive/My Drive/surgeon_profiling_fingerprint/data/Suturing/transcriptions/Suturing_I005.txt


100%|██████████| 78/78 [20:34<00:00, 15.83s/it]

  📽️ Extracted 756 frames from /content/drive/My Drive/surgeon_profiling_fingerprint/data/Suturing/video/Suturing_I005_capture2.avi
  Gesture label ranges (start–end–G):
    (73, 151, 0)
    (152, 241, 4)
    (242, 518, 7)
    (519, 658, 1)
    (659, 1198, 2)
  First 20 frame indices: [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]
  First 20 gesture labels: [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15  0  0  0  0  0]
  ✅ Appended 151 sequences for Suturing_I005_capture2.avi
    First gesture seq: [15 15 15 15 15]


Saved 10504 samples to /content/drive/My Drive/surgeon_profiling_fingerprint/data/jigsaws_full_processed.npz


In [16]:
import numpy as np

data = np.load("/content/drive/My Drive/surgeon_profiling_fingerprint/data/jigsaws_full_processed.npz")
print("Keys in file:", data.files)

for i in range(12):  # first 3 samples
    print(f"Sample {i}:")
    print("  gesture_seq:", data["gesture_seq"][i])
    print("  surgeon_id:", data["surgeon_id"][i])
    print("  vision_feat[:5]:", data["vision_feat"][i][:5])
    print("  lang_feat[:5]:", data["lang_feat"][i][:5])
    print()


Exception ignored in: <function NpzFile.__del__ at 0x7c373a996ca0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/numpy/lib/_npyio_impl.py", line 226, in __del__
  File "/usr/local/lib/python3.11/dist-packages/numpy/lib/_npyio_impl.py", line 221, in close
OSError: [Errno 107] Transport endpoint is not connected


Keys in file: ['vision_feat', 'lang_feat', 'gesture_seq', 'surgeon_id']
Sample 0:
  gesture_seq: [15 15 15 15 15]
  surgeon_id: 0
  vision_feat[:5]: [-2.7575073  1.1349099  0.8448879  1.5911711  3.8013864]
  lang_feat[:5]: [-0.20973833  0.23947342 -0.40593806 -0.03102328 -0.32488313]

Sample 1:
  gesture_seq: [15 15 15 15 15]
  surgeon_id: 0
  vision_feat[:5]: [-2.6374202   1.2563782   0.78266704  1.7316214   3.7721157 ]
  lang_feat[:5]: [-0.20973833  0.23947342 -0.40593806 -0.03102328 -0.32488313]

Sample 2:
  gesture_seq: [15 15 15 15 15]
  surgeon_id: 0
  vision_feat[:5]: [-2.673118   1.2887579  0.7227253  1.747577   3.741649 ]
  lang_feat[:5]: [-0.20973833  0.23947342 -0.40593806 -0.03102328 -0.32488313]

Sample 3:
  gesture_seq: [15  0  0  0  0]
  surgeon_id: 0
  vision_feat[:5]: [-2.3915565  0.922386   1.3196617  2.4087079  4.4498568]
  lang_feat[:5]: [-0.20973833  0.23947342 -0.40593806 -0.03102328 -0.32488313]

Sample 4:
  gesture_seq: [0 0 0 0 0]
  surgeon_id: 0
  vision_feat[